<a href="https://colab.research.google.com/github/jorgfernandosantana/Jorge_Santana/blob/main/C%C3%B3digo%20Garcia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q nltk scikit-learn

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Adiciona a linha para baixar o recurso 'punkt_tab'

from google.colab import files
uploaded = files.upload()

nome_arquivo = list(uploaded.keys())[0]
df = pd.read_csv(nome_arquivo, encoding='latin1')  # ou use utf-8 se necessário

def preprocess(text):
    stop_words = set(stopwords.words('english'))
    # Remove a especificação do language='english' na função word_tokenize
    # O recurso 'punkt_tab' parece ser usado quando a linguagem é especificada.
    # A remoção permite que NLTK use o 'punkt' baixado sem a necessidade explícita de 'punkt_tab'.
    tokens = word_tokenize(str(text).lower())
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

df.dropna(subset=['Title', 'Article'], inplace=True)
df['Processed_Article'] = df['Article'].apply(preprocess)

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Processed_Article'])

def recomendar_por_titulo(titulo, top_n=5):
    if titulo not in df['Title'].values:
        print(f"O título '{titulo}' não foi encontrado no dataset.")
        return

    idx = df[df['Title'] == titulo].index[0]
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]

    print(f"\nArtigos semelhantes a: '{titulo}':\n")
    for i in similar_indices:
        print(f"- {df.iloc[i]['Title']} (similaridade: {cosine_sim[i]:.2f})")

titulo_input = input("Digite o título do artigo: ")
recomendar_por_titulo(titulo_input)